In [1]:
from cryptolib import repeat_xor as xor
BLOCK_SIZE = 32

# Nothing up my sleeve numbers (ref: Dual_EC_DRBG P-256 coordinates)
W = [0x6b17d1f2, 0xe12c4247, 0xf8bce6e5, 0x63a440f2, 0x77037d81, 0x2deb33a0, 0xf4a13945, 0xd898c296]
X = [0x4fe342e2, 0xfe1a7f9b, 0x8ee7eb4a, 0x7c0f9e16, 0x2bce3357, 0x6b315ece, 0xcbb64068, 0x37bf51f5]
Y = [0xc97445f4, 0x5cdef9f0, 0xd3e05e1e, 0x585fc297, 0x235b82b5, 0xbe8ff3ef, 0xca67c598, 0x52018192]
Z = [0xb28ef557, 0xba31dfcb, 0xdd21ac46, 0xe2a91e3c, 0x304f44cb, 0x87058ada, 0x2cb81515, 0x1e610046]

# Lets work with bytes instead!
W_bytes = b''.join([x.to_bytes(4,'big') for x in W])
X_bytes = b''.join([x.to_bytes(4,'big') for x in X])
Y_bytes = b''.join([x.to_bytes(4,'big') for x in Y])
Z_bytes = b''.join([x.to_bytes(4,'big') for x in Z])

def pad(data):
    padding_len = (BLOCK_SIZE - len(data)) % BLOCK_SIZE
    return data + bytes([padding_len]*padding_len)

def blocks(data):
    return [data[i:(i+BLOCK_SIZE)] for i in range(0,len(data),BLOCK_SIZE)]

def rotate_left(data, x):
    return data[x:] + data[:x]

def rotate_right(data, x):
    return data[-x:] + data[:-x]

def scramble_block(block):
    for _ in range(40):
        block = xor(W_bytes, block)
        block = rotate_left(block, 6)
        block = xor(X_bytes, block)
        block = rotate_right(block, 17)
    return block

def unscramble_block(block):
    for _ in range(40):
        block = rotate_left(block, 17)
        block = xor(X_bytes, block)
        block = rotate_right(block, 6)
        block = xor(W_bytes, block)
    return block

def cryptohash(msg):
    initial_state = xor(Y_bytes, Z_bytes)
    msg_padded = pad(msg)
    msg_blocks = blocks(msg_padded)
    for i,b in enumerate(msg_blocks):
        mix_in = scramble_block(b)
        for _ in range(i):
            mix_in = rotate_right(mix_in, i+11)
            mix_in = xor(mix_in, X_bytes)
            mix_in = rotate_left(mix_in, i+6)
        initial_state = xor(initial_state,mix_in)
    return initial_state.hex()
def generate_collision(msg: str):
    assert len(msg) >= 32, "msg must be at least 32 bytes long for collision to exist"
    initial_state = b'{\xfa\xb0\xa3\xe6\xef&;\x0e\xc1\xf2X\xba\xf6\xdc\xab\x13\x14\xc6~9\x8ay5\xe6\xdf\xd0\x8dL`\x81\xd4'
    msg1 = msg.encode()
    msg2 = unscramble_block(xor(initial_state, bytes.fromhex(cryptohash(msg1))))
    return msg1, msg2
msg1, msg2 = generate_collision("Howdy! I'm Flowey the flower! You're new to the underground, aren'tcha? Golly, you must be so confused. Someone ought to teach you how things work around here! I guess little old me will have to do. Ready? Here we go!")
print(msg1, cryptohash(msg1))
print(msg2, cryptohash(msg2))

b"Howdy! I'm Flowey the flower! You're new to the underground, aren'tcha? Golly, you must be so confused. Someone ought to teach you how things work around here! I guess little old me will have to do. Ready? Here we go!" 348701183c9dbd154166ae27009c235f868a3b20afc3bcc426c890109c19a075
b'\x90\xcf\xc6\x12\x1agl\xca\x12\xd1\xdd\x07\x8b@-\xd2\xde\xee\xb3\xb37\xf8\xf1\tA\xeb\xb0\xe9\xc8\xbfx\xd3' 348701183c9dbd154166ae27009c235f868a3b20afc3bcc426c890109c19a075


In [2]:
from pwnlib.tubes.remote import remote
from cryptolib import json_send, json_recv, repeat_xor
r = remote("socket.cryptohack.org", 13405)
r.recvuntil(b"Please send two hex encoded messages m1, m2 formatted in JSON: ")
json_send(r, {"m1": msg1.hex(), "m2": msg2.hex()})
json_recv(r)
# r.recvline()
# json_send(r, {"document": open("message2.bin", 'rb').read().hex()})
# r.recvline()

{'flag': "Oh no! Looks like we have some more work to do... As promised, here's your flag: crypto{Always_add_padding_even_if_its_a_whole_block!!!}"}